In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('apache-practise').getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
access_key = ''
secret_key = ''
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", access_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret_key)

# If you are using Auto Loader file notification mode to load files, provide the AWS Region ID.
aws_region = "us-east-1"
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3." + aws_region + ".amazonaws.com")

df = spark.read.csv('s3a://outside-aws/employees_sample_data.csv',inferSchema=True,header=True)
df.show()




+-----------+----------+---------+-----+--------------+------+
|employee_id|first_name|last_name|p_age|    department|salary|
+-----------+----------+---------+-----+--------------+------+
|          1|      John|      Doe|   30|            HR| 50000|
|          2|      Jane|    Smith|   28|data engineeer| 70000|
|          3|     Alice|  Johnson|   35|     Marketing| 60000|
|          4|       Bob|    Brown|   40|        python| 55000|
|          5|   Charlie|    Davis|   25|          java| 65000|
|          6|     dilip|    bairi|   25| data engineer| 80000|
|          7|   heamnth|  sukumar|   24|    salesforce| 85000|
|          8|    srikar|   ambati|   24| data engineer| 70000|
|          9|    hredee|anishetty|   24|          jave| 60000|
|         10|   snehith|     puli|   24|        python| 90000|
+-----------+----------+---------+-----+--------------+------+



In [0]:
df=df.replace("jave","java",subset=["department"])
df=df.replace("data engineeer","data engineer",subset=["department"])

In [0]:
from pyspark.sql.functions import sum, avg

# Group by department and calculate the total and average salary
total_and_avg = df.groupBy("department").agg(
    sum("salary").alias("total_salary_by_department"),
    avg("salary").alias("average_salary_in_department")
)

# Show the result
total_and_avg.show()


+-------------+--------------------------+----------------------------+
|   department|total_salary_by_department|average_salary_in_department|
+-------------+--------------------------+----------------------------+
|   salesforce|                     85000|                     85000.0|
|           HR|                     50000|                     50000.0|
|data engineer|                    220000|           73333.33333333333|
|    Marketing|                     60000|                     60000.0|
|         java|                    125000|                     62500.0|
|       python|                    145000|                     72500.0|
+-------------+--------------------------+----------------------------+



In [0]:
from pyspark.sql.functions import col
df=df.withColumn("bonus",col("salary")*0.30)
df.show()

+-----------+----------+---------+-----+-------------+------+-------+
|employee_id|first_name|last_name|p_age|   department|salary|  bonus|
+-----------+----------+---------+-----+-------------+------+-------+
|          1|      John|      Doe|   30|           HR| 50000|15000.0|
|          2|      Jane|    Smith|   28|data engineer| 70000|21000.0|
|          3|     Alice|  Johnson|   35|    Marketing| 60000|18000.0|
|          4|       Bob|    Brown|   40|       python| 55000|16500.0|
|          5|   Charlie|    Davis|   25|         java| 65000|19500.0|
|          6|     dilip|    bairi|   25|data engineer| 80000|24000.0|
|          7|   heamnth|  sukumar|   24|   salesforce| 85000|25500.0|
|          8|    srikar|   ambati|   24|data engineer| 70000|21000.0|
|          9|    hredee|anishetty|   24|         java| 60000|18000.0|
|         10|   snehith|     puli|   24|       python| 90000|27000.0|
+-----------+----------+---------+-----+-------------+------+-------+



In [0]:
df=df.withColumn("salary_after_bonus", col("salary")+col("bonus"))
df.show()

+-----------+----------+---------+-----+-------------+------+-------+------------------+
|employee_id|first_name|last_name|p_age|   department|salary|  bonus|salary_after_bonus|
+-----------+----------+---------+-----+-------------+------+-------+------------------+
|          1|      John|      Doe|   30|           HR| 50000|15000.0|           65000.0|
|          2|      Jane|    Smith|   28|data engineer| 70000|21000.0|           91000.0|
|          3|     Alice|  Johnson|   35|    Marketing| 60000|18000.0|           78000.0|
|          4|       Bob|    Brown|   40|       python| 55000|16500.0|           71500.0|
|          5|   Charlie|    Davis|   25|         java| 65000|19500.0|           84500.0|
|          6|     dilip|    bairi|   25|data engineer| 80000|24000.0|          104000.0|
|          7|   heamnth|  sukumar|   24|   salesforce| 85000|25500.0|          110500.0|
|          8|    srikar|   ambati|   24|data engineer| 70000|21000.0|           91000.0|
|          9|    hred

In [0]:
from pyspark.sql.functions import when
df=df.withColumn("Tax_dedected_after_salary_bonous",when(col("salary_after_bonus")>100000,col("salary_after_bonus")*0.15).otherwise(col("salary_after_bonus")*0.10))
df.show()

+-----------+----------+---------+-----+-------------+------+-------+------------------+------------+--------------------------------+
|employee_id|first_name|last_name|p_age|   department|salary|  bonus|salary_after_bonus|Tax_dedected|Tax_dedected_after_salary_bonous|
+-----------+----------+---------+-----+-------------+------+-------+------------------+------------+--------------------------------+
|          1|      John|      Doe|   30|           HR| 50000|15000.0|           65000.0|      5000.0|                          6500.0|
|          2|      Jane|    Smith|   28|data engineer| 70000|21000.0|           91000.0|      7000.0|                          9100.0|
|          3|     Alice|  Johnson|   35|    Marketing| 60000|18000.0|           78000.0|      6000.0|                          7800.0|
|          4|       Bob|    Brown|   40|       python| 55000|16500.0|           71500.0|      5500.0|                          7150.0|
|          5|   Charlie|    Davis|   25|         java| 

In [0]:
import pandas as pd
df=df.withColumn("permenant_employee",when(col("p_age")>=35,"permanent employee").otherwise("contracted based"))
da=df.toPandas()
da

,employee_id,first_name,last_name,p_age,department,salary,bonus,salary_after_bonus,Tax_dedected,Tax_dedected_after_salary_bonous,permenant_employee
0,1,John,Doe,30,HR,50000,15000.0,65000.0,5000.0,6500.0,contracted based
1,2,Jane,Smith,28,data engineer,70000,21000.0,91000.0,7000.0,9100.0,contracted based
2,3,Alice,Johnson,35,Marketing,60000,18000.0,78000.0,6000.0,7800.0,permanent employee
3,4,Bob,Brown,40,python,55000,16500.0,71500.0,5500.0,7150.0,permanent employee
4,5,Charlie,Davis,25,java,65000,19500.0,84500.0,6500.0,8450.0,contracted based
5,6,dilip,bairi,25,data engineer,80000,24000.0,104000.0,8000.0,15600.0,contracted based
6,7,heamnth,sukumar,24,salesforce,85000,25500.0,110500.0,8500.0,16575.0,contracted based
7,8,srikar,ambati,24,data engineer,70000,21000.0,91000.0,7000.0,9100.0,contracted based
8,9,hredee,anishetty,24,java,60000,18000.0,78000.0,6000.0,7800.0,contracted based
9,10,snehith,puli,24,python,90000,27000.0,117000.0,9000.0,17550.0,contracted based


In [0]:
# Create a temporary view called "employees"
df.createOrReplaceTempView("employees")
# Run SQL query to select specific columns and filter data
result = spark.sql("""
                   SELECT first_name, last_name, p_age FROM employees WHERE p_age > 30 ORDER BY p_age DESC
                   """)

# Show the query result
result.show()



+----------+---------+-----+
|first_name|last_name|p_age|
+----------+---------+-----+
|       Bob|    Brown|   40|
|     Alice|  Johnson|   35|
+----------+---------+-----+



In [0]:
%sql
select department, sum(salary) from employees group by department order by SUM(salary) DESC

department,sum(salary)
data engineer,220000
python,145000
java,125000
salesforce,85000
Marketing,60000
HR,50000
